# Define probability distribution by using Distribution API


Distribution API document: https://docs.pixyz.io/en/latest/distributions.html

In [1]:
from __future__ import print_function
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np

torch.manual_seed(1)

In [2]:
from pixyz.utils import print_latex

## 1. Define probability distribution without Deep Neural Networks

### 1.1 Define a simple probability distribution

To define a Gaussian distribution, we need to
1. import `pixyz.distributions.Normal` class
2. set mean (loc) & variance(scale)

In [3]:
from pixyz.distributions import Normal

x_dim = 50
p1_nor_x = Normal(loc=torch.tensor(0.), scale=torch.tensor(1.), var=['x'], features_shape=[x_dim], name='p_{1}')

We define a variable name in `var` (in this example, variable name is x), and specify the number of dimensions in `features_shape`(in this example, the number of dimensions 50(x_dim)). 

We can check defined probability distribution's information.

In [4]:
print(p1_nor_x.distribution_name) 
print(p1_nor_x.prob_text)

Normal
p_{1}(x)


In `distribution_name`, we can check the name of distribution.

In `prob_text`, we can check the probability distribution in text and random variable shows `var` defined above(x).

By printing p1_nor_x，we can overview the features of the distribution.

In [5]:
print(p1_nor_x)

Distribution:
  p_{1}(x)
Network architecture:
  Normal(
    name=p_{1}, distribution_name=Normal,
    var=['x'], cond_var=[], input_var=[], features_shape=torch.Size([50])
    (loc): torch.Size([1, 50])
    (scale): torch.Size([1, 50])
  )


We can check the defined probability distribution in LaTeX format.  
Note: We use external library SymPy(https://www.sympy.org/en/index.html) for outputting LaTeX format. The order of the terms in the formula can be changed(but not affecting the result) due to the SymPy.

In [6]:
print_latex(p1_nor_x)

<IPython.core.display.Math object>

We can sample from the defined probability distribution by `.sample()`.

In [7]:
p1_nor_x_samples = p1_nor_x.sample()
print(p1_nor_x_samples)
print('--------------------------------------------------------------------------')
print(p1_nor_x_samples["x"])

{'x': tensor([[-1.5256, -0.7502, -0.6540, -1.6095, -0.1002, -0.6092, -0.9798, -1.6091,
         -0.7121,  0.3037, -0.7773, -0.2515, -0.2223,  1.6871,  0.2284,  0.4676,
         -0.6970, -1.1608,  0.6995,  0.1991,  0.8657,  0.2444, -0.6629,  0.8073,
          1.1017, -0.1759, -2.2456, -1.4465,  0.0612, -0.6177, -0.7981, -0.1316,
          1.8793, -0.0721,  0.0663, -0.4370,  0.7626,  0.4415,  1.1651,  2.0154,
          0.2152, -0.5242, -0.1860, -0.6446,  1.5392, -0.8696, -3.3312, -0.7479,
          1.1173,  0.2981]])}
--------------------------------------------------------------------------
tensor([[-1.5256, -0.7502, -0.6540, -1.6095, -0.1002, -0.6092, -0.9798, -1.6091,
         -0.7121,  0.3037, -0.7773, -0.2515, -0.2223,  1.6871,  0.2284,  0.4676,
         -0.6970, -1.1608,  0.6995,  0.1991,  0.8657,  0.2444, -0.6629,  0.8073,
          1.1017, -0.1759, -2.2456, -1.4465,  0.0612, -0.6177, -0.7981, -0.1316,
          1.8793, -0.0721,  0.0663, -0.4370,  0.7626,  0.4415,  1.1651,  2.0154

Output is dict type.

We can check specific variable's sampling output by specifying the variable name in output dict.

Sampling result is a PyTorch Tensor.

### 1.2 Define conditional probability distribution

Next, we define a conditional probability distribution using an example of a Gaussian distribution.

In Gaussian distribution, parameters are mean($\mu$) and variance($\sigma^2$). In this example, we define a Gaussian distribution conditioned by $\mu$.

$p(x|\mu_{var}) = \cal N(x; \mu=\mu_{var}, \sigma^2=1)$

We set conditional variables to `cond_var`.  
In this example, we set mu_var to a Gaussian distribution's mean, so in the distribution argument, we set  
- cond_var=['mu_var']  
- loc='mu_var'  

In [8]:
x_dim = 50
p1_nor_x__mu = Normal(loc='mu_var', scale=torch.tensor(1.), var=['x'], cond_var=['mu_var'], features_shape=[x_dim])

In [9]:
print(p1_nor_x__mu)
print_latex(p1_nor_x__mu)

Distribution:
  p(x|\mu_{var})
Network architecture:
  Normal(
    name=p, distribution_name=Normal,
    var=['x'], cond_var=['mu_var'], input_var=['mu_var'], features_shape=torch.Size([50])
    (scale): torch.Size([1, 50])
  )


<IPython.core.display.Math object>

We successfully define a Gaussian distribution whose mean conditioned by $\mu_{var}$.  
Let's try sampling x setting $\mu_{var}=0$.  
We set variable in sample method argument dict.  

In [10]:
p1_nor_x__mu.sample({"mu_var": 0})

{'mu_var': 0,
 'x': tensor([[-0.5962, -1.0055, -0.2106, -0.0075,  1.6734,  0.0103,  0.9837,  0.8793,
          -0.9962, -0.8313, -0.4610, -0.5601,  0.3956, -0.9823,  1.3264,  0.8547,
          -0.6540,  0.7317, -1.4344, -0.5008,  0.1716, -0.1600, -0.5047, -1.4746,
          -1.0412,  0.7323, -1.0483, -0.4709,  0.2911,  1.9907, -0.9247, -0.9301,
           0.8165, -0.9135,  0.2053,  0.3051,  0.5357, -0.4312,  0.1573,  1.2540,
           1.3275, -0.4954, -1.9804,  1.7986,  0.1018,  0.3400, -0.6447, -0.2870,
           3.3212, -0.4021]])}

Next, we assume $\mu_{var}$ itsself is conditioned by some probability distribution.  
We assume $\mu_{var}$ follows Bernoulli distribution.  
$p(\mu_{var}) = \cal B(\mu_{var};p=0.3)$

In [11]:
from pixyz.distributions import Bernoulli
p2_ber_mu = Bernoulli(probs=torch.tensor(0.3), var=['mu_var'], features_shape=[x_dim])

In [12]:
print(p2_ber_mu)
print(p2_ber_mu.sample())
print_latex(p2_ber_mu)

Distribution:
  p(\mu_{var})
Network architecture:
  Bernoulli(
    name=p, distribution_name=Bernoulli,
    var=['mu_var'], cond_var=[], input_var=[], features_shape=torch.Size([50])
    (probs): torch.Size([1, 50])
  )
{'mu_var': tensor([[0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
         1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])}


<IPython.core.display.Math object>

In Pixyz Distribution API, joint distribution can be defined by multiplying distributions.  
Let's define the joint distribution $p(x, \mu_{var})$ multiplying $p(\mu_{var})$ and $p(x|\mu_{var})$.  
$p(x, \mu_{var}) = p(x|\mu_{var}) p(\mu_{var})$


In [13]:
p_joint_mu_x = p1_nor_x__mu * p2_ber_mu
print(p_joint_mu_x) 
print_latex(p_joint_mu_x)

Distribution:
  p(x,\mu_{var}) = p(x|\mu_{var})p(\mu_{var})
Network architecture:
  Bernoulli(
    name=p, distribution_name=Bernoulli,
    var=['mu_var'], cond_var=[], input_var=[], features_shape=torch.Size([50])
    (probs): torch.Size([1, 50])
  )
  Normal(
    name=p, distribution_name=Normal,
    var=['x'], cond_var=['mu_var'], input_var=['mu_var'], features_shape=torch.Size([50])
    (scale): torch.Size([1, 50])
  )


<IPython.core.display.Math object>

Sampling from joint distributions also can be done using `.sample()`.  
All variables and values are output in dict type.

In [14]:
p_joint_mu_x.sample()

{'mu_var': tensor([[1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
          0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1.]]),
 'x': tensor([[ 3.6415, -0.9624,  0.7924, -1.3889,  1.0127, -0.8734,  1.7997,  1.2824,
           1.6604,  0.2717,  0.1913,  0.1267,  0.5707,  0.8652,  0.3437,  0.3718,
           0.1444,  1.7772, -2.3381,  0.1709,  1.1661,  1.4787,  0.2676,  0.7561,
          -0.5873, -2.0619,  0.4305,  0.3377, -0.3438, -0.6172,  2.2530, -0.0514,
          -1.0257,  0.5213, -2.3065,  1.6037,  0.1794,  0.1447,  0.6411,  0.4793,
           0.7617, -0.3542, -0.2693,  2.3120, -0.8920, -0.7529, -0.0573,  2.2000,
           0.9912,  0.9414]])}

## 2. Define probability distribution with Deep Neural Networks

In this section, we master how to define probability distributions with deep neural networks.

For example, a Gaussian distribution's mean $\mu$ and variance $\sigma^2$ can be parameterized by $\theta$ like this way:  
$\mu=f(x;\theta)$  
$\sigma^2=g(x;\theta)$  
$f(x;\theta)$ and $g(x;\theta)$ stands for deep neural networks.

${\cal N}(\mu=f(x;\theta),\sigma^2=g(x;\theta))$  

Let's define
$p(a) = {\cal N}(a; \mu=f(x;\theta),\sigma^2=g(x;\theta))$.

In Pixyz, we can define this kind of probability distribution by inheriting `pixyz.distributions` class.

In [15]:
from pixyz.distributions import Normal
a_dim = 20

class ProbNorAgivX(Normal):
    """
    Probability distrituion Normal A given X
    p(a) = {\cal N}(a; \mu=f(x;\theta),\sigma^2=g(x;\theta)
    loc and scale are parameterized by theta given x
    """
    def __init__(self):
        super(ProbNorAgivX, self).__init__(cond_var=['x'], var=['a'])
        
        self.fc1 = nn.Linear(x_dim, 10)
        self.fc_loc = nn.Linear(10, a_dim)
        self.fc_scale = nn.Linear(10, a_dim)
        
    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        return {'loc': self.fc_loc(h1), 'scale': F.softplus(self.fc_scale(h1))}
p_nor_a__x = ProbNorAgivX()

First, clarify that the parameters of the Gaussian distribution are defiend by the deep neural networks by inheriting the Gaussian distribution.  


Next, describe the neural network architecture in constructor just like when we are writing PyTorch.


The only difference is that we set `var` and `cond_var` to `super()` args.

We set the output variable name to `var`. And we set the NN's input variable name to `cond_var`. We regard this input variable as conditioning variable of this Gaussian distribution.  

In forward method, there are two caveats．

* `forward()` args's variable name and variable number shold be the same as those set in `cond_var`. For example, if we set `cond_var=["x", "y"]`, we must set `forward(self, x, y)`
* return output should be parameters of the distribution as dict type. In this Gaussian distribution example, we set parameters `loc` and `scale` in dict type.

Finally, make an instance of defined probability distribution.

Let's check the distribution features by `print()`.

In [16]:
print(p_nor_a__x)
print_latex(p_nor_a__x)

Distribution:
  p(a|x)
Network architecture:
  ProbNorAgivX(
    name=p, distribution_name=Normal,
    var=['a'], cond_var=['x'], input_var=['x'], features_shape=torch.Size([])
    (fc1): Linear(in_features=50, out_features=10, bias=True)
    (fc_loc): Linear(in_features=10, out_features=20, bias=True)
    (fc_scale): Linear(in_features=10, out_features=20, bias=True)
  )


<IPython.core.display.Math object>

This distribution is conditioned by x like we set `cond_var` when defining constructor.

We can do sampling by `.sample()` but there is one point to be care about.

This is conditional probability distribution, so we explicitly input conditional variable when sampling.

We prepare `x_samples` and set `x_samples` to conditional variable `x`.

In [17]:
x_samples = torch.Tensor([[-0.3030, -1.7618,  0.6348, -0.8044, -1.0371, -1.0669, -0.2085,
         -0.2155,  2.2952,  0.6749,  1.7133, -1.7943, -1.5208,  0.9196,
         -0.5484, -0.3472,  0.4730, -0.4286,  0.5514, -1.5474,  0.7575,
         -0.4068, -0.1277,  0.2804,  1.7460,  1.8550, -0.7064,  2.5571,
          0.7705, -1.0739, -0.2015, -0.5603, -0.6240, -0.9773, -0.1637,
         -0.3582, -0.0594, -2.4919,  0.2423,  0.2883, -0.1095,  0.3126,
         -0.3417,  0.9473,  0.6223, -0.4481, -0.2856,  0.3880, -1.1435,
         -0.6512]])

In [18]:
p_nor_a__x_samples = p_nor_a__x.sample({'x': x_samples})
print(p_nor_a__x_samples)
print(p_nor_a__x_samples['a'])
print(p_nor_a__x_samples['x'])

{'x': tensor([[-0.3030, -1.7618,  0.6348, -0.8044, -1.0371, -1.0669, -0.2085, -0.2155,
          2.2952,  0.6749,  1.7133, -1.7943, -1.5208,  0.9196, -0.5484, -0.3472,
          0.4730, -0.4286,  0.5514, -1.5474,  0.7575, -0.4068, -0.1277,  0.2804,
          1.7460,  1.8550, -0.7064,  2.5571,  0.7705, -1.0739, -0.2015, -0.5603,
         -0.6240, -0.9773, -0.1637, -0.3582, -0.0594, -2.4919,  0.2423,  0.2883,
         -0.1095,  0.3126, -0.3417,  0.9473,  0.6223, -0.4481, -0.2856,  0.3880,
         -1.1435, -0.6512]]), 'a': tensor([[-1.7231e-01, -5.0856e-01,  1.3573e+00, -7.1246e-01,  3.8644e-01,
          1.1225e+00,  1.4864e-01,  6.8819e-02, -5.6884e-01, -2.4427e+00,
          1.2279e-03, -9.0337e-01,  5.3217e-02,  6.0509e-01, -3.8033e-01,
          6.5706e-02, -2.3049e-01,  3.4607e-01,  2.6745e-02, -3.9659e-01]])}
tensor([[-1.7231e-01, -5.0856e-01,  1.3573e+00, -7.1246e-01,  3.8644e-01,
          1.1225e+00,  1.4864e-01,  6.8819e-02, -5.6884e-01, -2.4427e+00,
          1.2279e-03, -9.0

Output contains samples of a and x.

a is samples by `.sample()` from the distribution and x is feeded `x_samples`.

### Next Tutorial
02-LossAPITutorial.ipynb